In [2]:
# !pip install --upgrade langchain accelerate transformers sentencepiece accelerate bitsandbytes scipy
# !pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cpu
!export PYTORCH_ENABLE_MPS_FALLBACK=1

In [4]:
import torch
from langchain.chains import (
    LLMChain
)
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [5]:
# Globals
repo_id = 'tiiuae/falcon-40b-instruct'

quantization_config = BitsAndBytesConfig(llm_int8_enable_fp32_cpu_offload=True)

tokenizer = AutoTokenizer.from_pretrained(repo_id)
model = AutoModelForCausalLM.from_pretrained(
    repo_id,
    device_map="auto",
    trust_remote_code=True,
    load_in_8bit=True,
    quantization_config=quantization_config,
    offload_folder="/tmp/offload/falcon40b"
)

generate_text = pipeline(
    "text-generation",
    max_length=1000,
    model=model,
    torch_dtype=torch.float16,
    tokenizer=tokenizer,
    trust_remote_code=True,
    device_map="auto",
    model_kwargs={
        'pad_token_id': tokenizer.eos_token_id,
        'num_return_sequences': 1,
        'top_k': 10,
        'do_sample': True
    }
)

transformer_pipeline = HuggingFacePipeline(pipeline=generate_text)

Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

In [6]:
prompt = PromptTemplate.from_template(
    "Suggest a good company name to describe a company that makes {product}."
)

chain = LLMChain(
    llm=transformer_pipeline,
    prompt=prompt
)

In [7]:
product = "Artistic and visually stunning hand bags."
result = chain.run(product)
print(result)

/Users/bsantanna/miniforge3/envs/ml/lib/python3.11/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



How about 'Artistic Handbags'? It's simple, straightforward, and accurately describes the company's products.
